# Understanding movie success through rotten tomatoes and TMDB

**Author:** David Mwai Gathimba

## Project Overview

In this project, we are going to provide a data-driven report to Microsoft's new movie studio on the success factors of movies using data from Rotten Tomatoes and TMDB. One of the basic business questions for Microsoft is figuring out which types of movies are likely to do the best in theaters—since the company has virtually no experience in the world of making motion pictures. To look for the trends and correlations, we use datasets of movie reviews, movie ratings, and movie popularity metrics. We walk through data cleaning, exploratory data analysis, and visualization to expose leading indicators. The results also show that success is associated with key factors including genre, review scores, and popularity. According to the results, we suggest that Microsoft should make movies with good ratings and movements in popular genres to get the highest possible performance in the competitive movie circle.


## Business Problem

Microsoft is in the process of launching a new film studio, entering into the competitive film industry, which they have no experience with, thus the reason they are leveraging data to do so. The business problem we are trying to solve is to find out the most important features which influence the revenue and popularity of movies. In this post, we will answer a couple of data questions on the observation: What types of movies are doing well at the box office scene genre-wise? Do Rotten Tomatoes film scores affect box office revenue? How does the popularity of a movie on TMDB relate to its box-office performance? What difference would it make to a movie whether these attributes like run time or period of a release play a vital role in determining the success of a movie? So what about those insights for Microsoft, to inform their entry into movie production in a strategic way?
These queries are designed to directly correlate to quantitative properties of movies, and are looking for patterns or trends the data might reveal, to advise Microsoft at the strategy level. Having insight into these factors is essential for Microsoft to mitigate costs, allocate resources efficiently, and ensure its movie studio has the best chance of success in an already crowded market. In addition to genre popularity and the effect of reviews and popularity, we will use 10-fold cross-validation to foresee potential box office performance that could act as a guide star for Microsoft, which tries to be smarter about producing data-driven production decisions.

## Data Understanding

In [1]:
# import packages
import pandas as pd
import csv
import numpy as np
import matplotlib as plt

In [2]:
# load data
df1 = pd.read_csv('Data/bom.movie_gross.csv')
df2 = pd.read_csv('Data/tmdb.movies.csv')

print(df1.head())
print(df2.head())

                                         title studio  domestic_gross  \
0                                  Toy Story 3     BV     415000000.0   
1                   Alice in Wonderland (2010)     BV     334200000.0   
2  Harry Potter and the Deathly Hallows Part 1     WB     296000000.0   
3                                    Inception     WB     292600000.0   
4                          Shrek Forever After   P/DW     238700000.0   

  foreign_gross  year  
0     652000000  2010  
1     691300000  2010  
2     664300000  2010  
3     535700000  2010  
4     513900000  2010  
   Unnamed: 0            genre_ids     id original_language  \
0           0      [12, 14, 10751]  12444                en   
1           1  [14, 12, 16, 10751]  10191                en   
2           2        [12, 28, 878]  10138                en   
3           3      [16, 35, 10751]    862                en   
4           4        [28, 878, 12]  27205                en   

                                 origi

In [3]:
df1.shape

(3387, 5)

In [4]:
df2.shape

(26517, 10)

In [5]:
# explore data
df1.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [6]:
df2.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


## Data Preparation

In [7]:
# merge the dataframes
df4=pd.merge(df1,df2, on='title', how="right")
df4.shape

(26517, 14)

In [8]:
df4.shape

(26517, 14)

In [9]:
# missing data
missing_data1 = df4.isna()
print(df1.isnull().sum())

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64


In [10]:
df4.keys

<bound method NDFrame.keys of                                               title studio  domestic_gross  \
0      Harry Potter and the Deathly Hallows: Part 1    NaN             NaN   
1                          How to Train Your Dragon   P/DW     217600000.0   
2                                        Iron Man 2   Par.     312400000.0   
3                                         Toy Story    NaN             NaN   
4                                         Inception     WB     292600000.0   
...                                             ...    ...             ...   
26512                         Laboratory Conditions    NaN             NaN   
26513                               _EXHIBIT_84xxx_    NaN             NaN   
26514                                  The Last One    NaN             NaN   
26515                                  Trailer Made    NaN             NaN   
26516                                    The Church    NaN             NaN   

      foreign_gross    year  Unna

In [11]:
rows_with_missing = df4[df4.isnull().any(axis=1)]
print(rows_with_missing)

                                                   title studio  \
0           Harry Potter and the Deathly Hallows: Part 1    NaN   
3                                              Toy Story    NaN   
5      Percy Jackson & the Olympians: The Lightning T...    NaN   
6                                                 Avatar    NaN   
10                                           Toy Story 2    NaN   
...                                                  ...    ...   
26512                              Laboratory Conditions    NaN   
26513                                    _EXHIBIT_84xxx_    NaN   
26514                                       The Last One    NaN   
26515                                       Trailer Made    NaN   
26516                                         The Church    NaN   

       domestic_gross foreign_gross  year  Unnamed: 0          genre_ids  \
0                 NaN           NaN   NaN           0    [12, 14, 10751]   
3                 NaN           NaN   NaN  

In [12]:
df4.columns

Index(['title', 'studio', 'domestic_gross', 'foreign_gross', 'year',
       'Unnamed: 0', 'genre_ids', 'id', 'original_language', 'original_title',
       'popularity', 'release_date', 'vote_average', 'vote_count'],
      dtype='object')

In [13]:
df4

,title,studio,domestic_gross,foreign_gross,year,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,vote_average,vote_count
0,Harry Potter and the Deathly Hallows: Part 1,NaN,NaN,NaN,NaN,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,7.7,10788
1,How to Train Your Dragon,P/DW,217600000.0,277300000,2010.0,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,7.7,7610
2,Iron Man 2,Par.,312400000.0,311500000,2010.0,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,6.8,12368
3,Toy Story,NaN,NaN,NaN,NaN,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,7.9,10174
4,Inception,WB,292600000.0,535700000,2010.0,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,8.3,22186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26512,Laboratory Conditions,NaN,NaN,NaN,NaN,26512,"[27, 18]",488143,en,Laboratory Conditions,0.600,2018-10-13,0.0,1
26513,_EXHIBIT_84xxx_,NaN,NaN,NaN,NaN,26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.600,2018-05-01,0.0,1
26514,The Last One,NaN,NaN,NaN,NaN,26514,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,0.0,1
26515,Trailer Made,NaN,NaN,NaN,NaN,26515,"[10751, 12, 28]",366854,en,Trailer Made,0.600,2018-06-22,0.0,1


In [18]:
df4 = df4.drop(columns=["Unnamed: 0","original_language","original_title"])
df4

,title,studio,domestic_gross,foreign_gross,year,genre_ids,id,popularity,release_date,vote_average,vote_count
0,Harry Potter and the Deathly Hallows: Part 1,NaN,NaN,NaN,NaN,"[12, 14, 10751]",12444,33.533,2010-11-19,7.7,10788
1,How to Train Your Dragon,P/DW,217600000.0,277300000,2010.0,"[14, 12, 16, 10751]",10191,28.734,2010-03-26,7.7,7610
2,Iron Man 2,Par.,312400000.0,311500000,2010.0,"[12, 28, 878]",10138,28.515,2010-05-07,6.8,12368
3,Toy Story,NaN,NaN,NaN,NaN,"[16, 35, 10751]",862,28.005,1995-11-22,7.9,10174
4,Inception,WB,292600000.0,535700000,2010.0,"[28, 878, 12]",27205,27.920,2010-07-16,8.3,22186
...,...,...,...,...,...,...,...,...,...,...,...
26512,Laboratory Conditions,NaN,NaN,NaN,NaN,"[27, 18]",488143,0.600,2018-10-13,0.0,1
26513,_EXHIBIT_84xxx_,NaN,NaN,NaN,NaN,"[18, 53]",485975,0.600,2018-05-01,0.0,1
26514,The Last One,NaN,NaN,NaN,NaN,"[14, 28, 12]",381231,0.600,2018-10-01,0.0,1
26515,Trailer Made,NaN,NaN,NaN,NaN,"[10751, 12, 28]",366854,0.600,2018-06-22,0.0,1


In [19]:
df4.to_csv('moviedb.csv')

In [23]:
print(df4['popularity'].dtype)
print(df4['domestic_gross'].dtype)
print(df4['foreign_gross'].dtype)
# print(df4['popularity'].dtype)
# print(df4['popularity'].dtype)

float64
float64
object


In [25]:
# replace the NaN values with an integer (0) and change to integer
df4['foreign_gross'] = df4['foreign_gross'].astype(str).str.replace('NaN', 'np.nan', regex=False)
df4['foreign_gross'] = pd.to_numeric(df4['foreign_gross'], errors='coerce')
df4['foreign_gross'] = df4['foreign_gross'].fillna(0)


In [26]:
df4['popularity'] = df4['popularity'].astype(str).str.replace('NaN', 'np.nan', regex=False)
df4['popularity'] = pd.to_numeric(df4['popularity'], errors='coerce')
df4['popularity'] = df4['popularity'].fillna(0)




In [27]:
df4['domestic_gross'] = df4['domestic_gross'].astype(str).str.replace('NaN', 'np.nan', regex=False)
df4['domestic_gross'] = pd.to_numeric(df4['domestic_gross'], errors='coerce')
df4['domestic_gross'] = df4['domestic_gross'].fillna(0)

In [32]:

df4.popularity = df4.popularity.astype(int)
df4.popularity = df4.popularity.astype(int)
df4.foreign_gross = df4.foreign_gross.astype(int)

In [33]:
print(df4['popularity'].dtype)
print(df4['domestic_gross'].dtype)
print(df4['foreign_gross'].dtype)

int32
float64
int32


In [20]:
# change date format
df4['release_date'] = pd.to_datetime(df4['release_date'])
df4['release_date'].dtypes

dtype('<M8[ns]')

In [21]:
df4

,title,studio,domestic_gross,foreign_gross,year,genre_ids,id,popularity,release_date,vote_average,vote_count
0,Harry Potter and the Deathly Hallows: Part 1,NaN,NaN,NaN,NaN,"[12, 14, 10751]",12444,33.533,2010-11-19,7.7,10788
1,How to Train Your Dragon,P/DW,217600000.0,277300000,2010.0,"[14, 12, 16, 10751]",10191,28.734,2010-03-26,7.7,7610
2,Iron Man 2,Par.,312400000.0,311500000,2010.0,"[12, 28, 878]",10138,28.515,2010-05-07,6.8,12368
3,Toy Story,NaN,NaN,NaN,NaN,"[16, 35, 10751]",862,28.005,1995-11-22,7.9,10174
4,Inception,WB,292600000.0,535700000,2010.0,"[28, 878, 12]",27205,27.920,2010-07-16,8.3,22186
...,...,...,...,...,...,...,...,...,...,...,...
26512,Laboratory Conditions,NaN,NaN,NaN,NaN,"[27, 18]",488143,0.600,2018-10-13,0.0,1
26513,_EXHIBIT_84xxx_,NaN,NaN,NaN,NaN,"[18, 53]",485975,0.600,2018-05-01,0.0,1
26514,The Last One,NaN,NaN,NaN,NaN,"[14, 28, 12]",381231,0.600,2018-10-01,0.0,1
26515,Trailer Made,NaN,NaN,NaN,NaN,"[10751, 12, 28]",366854,0.600,2018-06-22,0.0,1


## Evaluation and Visualization
***

***

In [23]:
import requests

In [24]:
# get genre from TMDB movies for mapping
response = requests.get('https://api.themoviedb.org/3/genre/movie/list?api_key=104bebf4fc2218b241e2351d57278652')
print(response.status_code)

200


In [25]:
print(response.json())

{'genres': [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 80, 'name': 'Crime'}, {'id': 99, 'name': 'Documentary'}, {'id': 18, 'name': 'Drama'}, {'id': 10751, 'name': 'Family'}, {'id': 14, 'name': 'Fantasy'}, {'id': 36, 'name': 'History'}, {'id': 27, 'name': 'Horror'}, {'id': 10402, 'name': 'Music'}, {'id': 9648, 'name': 'Mystery'}, {'id': 10749, 'name': 'Romance'}, {'id': 878, 'name': 'Science Fiction'}, {'id': 10770, 'name': 'TV Movie'}, {'id': 53, 'name': 'Thriller'}, {'id': 10752, 'name': 'War'}, {'id': 37, 'name': 'Western'}]}


In [28]:
df4 = df4.drop(columns=["genre_name"])

In [29]:
genre_name = response.json()
genre_name

{'genres': [{'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'},
  {'id': 16, 'name': 'Animation'},
  {'id': 35, 'name': 'Comedy'},
  {'id': 80, 'name': 'Crime'},
  {'id': 99, 'name': 'Documentary'},
  {'id': 18, 'name': 'Drama'},
  {'id': 10751, 'name': 'Family'},
  {'id': 14, 'name': 'Fantasy'},
  {'id': 36, 'name': 'History'},
  {'id': 27, 'name': 'Horror'},
  {'id': 10402, 'name': 'Music'},
  {'id': 9648, 'name': 'Mystery'},
  {'id': 10749, 'name': 'Romance'},
  {'id': 878, 'name': 'Science Fiction'},
  {'id': 10770, 'name': 'TV Movie'},
  {'id': 53, 'name': 'Thriller'},
  {'id': 10752, 'name': 'War'},
  {'id': 37, 'name': 'Western'}]}

In [31]:
genre_name = pd.DataFrame(genre_name)

genre_name

,genres
0,"{'id': 28, 'name': 'Action'}"
1,"{'id': 12, 'name': 'Adventure'}"
2,"{'id': 16, 'name': 'Animation'}"
3,"{'id': 35, 'name': 'Comedy'}"
4,"{'id': 80, 'name': 'Crime'}"
5,"{'id': 99, 'name': 'Documentary'}"
6,"{'id': 18, 'name': 'Drama'}"
7,"{'id': 10751, 'name': 'Family'}"
8,"{'id': 14, 'name': 'Fantasy'}"
9,"{'id': 36, 'name': 'History'}"


In [44]:

genre_name = pd.DataFrame(genre_name)
genre_name

,genres
0,"{'': 28, 'name': 'Action'}"
1,"{'': 12, 'name': 'Adventure'}"
2,"{'': 16, 'name': 'Animation'}"
3,"{'': 35, 'name': 'Comedy'}"
4,"{'': 80, 'name': 'Crime'}"
5,"{'': 99, 'name': 'Documentary'}"
6,"{'': 18, 'name': 'Drama'}"
7,"{'': 10751, 'name': 'Family'}"
8,"{'': 14, 'name': 'Fantasy'}"
9,"{'': 36, 'name': 'History'}"


In [45]:
genre_name['genres'].astype(str).str.replace('{', '', regex=False)
genre_name['genres'].astype(str).str.replace('}', '', regex=False)
genre_name['genres'].astype(str).str.replace('name', '', regex=False)

0               {'': 28, '': 'Action'}
1            {'': 12, '': 'Adventure'}
2            {'': 16, '': 'Animation'}
3               {'': 35, '': 'Comedy'}
4                {'': 80, '': 'Crime'}
5          {'': 99, '': 'Documentary'}
6                {'': 18, '': 'Drama'}
7            {'': 10751, '': 'Family'}
8              {'': 14, '': 'Fantasy'}
9              {'': 36, '': 'History'}
10              {'': 27, '': 'Horror'}
11            {'': 10402, '': 'Music'}
12           {'': 9648, '': 'Mystery'}
13          {'': 10749, '': 'Romance'}
14    {'': 878, '': 'Science Fiction'}
15         {'': 10770, '': 'TV Movie'}
16            {'': 53, '': 'Thriller'}
17              {'': 10752, '': 'War'}
18             {'': 37, '': 'Western'}
Name: genres, dtype: object

In [46]:
genre_name = genre_name.to_dict(orient='list')

In [47]:
genre_mapping = {
    28: 'Action',
    12: 'Adventure',
    16: 'Animation',
    35: 'Comedy',
    80: 'Crime',
    99: 'Documentary',
    18: 'Drama',
    10751: 'Family',
    14: 'Fantasy',
    36: 'History', 
    27: 'Horror',
    10402: 'Music',
    9648: 'Mystery',
    10749: 'Romance',
    878: 'Science Fiction',
    10770: 'TV Movie',
    53: 'Thriller',
    10752: 'War',
    37: 'Western'
}



{'genres': ["{'': 28, 'name': 'Action'}",
  "{'': 12, 'name': 'Adventure'}",
  "{'': 16, 'name': 'Animation'}",
  "{'': 35, 'name': 'Comedy'}",
  "{'': 80, 'name': 'Crime'}",
  "{'': 99, 'name': 'Documentary'}",
  "{'': 18, 'name': 'Drama'}",
  "{'': 10751, 'name': 'Family'}",
  "{'': 14, 'name': 'Fantasy'}",
  "{'': 36, 'name': 'History'}",
  "{'': 27, 'name': 'Horror'}",
  "{'': 10402, 'name': 'Music'}",
  "{'': 9648, 'name': 'Mystery'}",
  "{'': 10749, 'name': 'Romance'}",
  "{'': 878, 'name': 'Science Fiction'}",
  "{'': 10770, 'name': 'TV Movie'}",
  "{'': 53, 'name': 'Thriller'}",
  "{'': 10752, 'name': 'War'}",
  "{'': 37, 'name': 'Western'}"]}

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have a DataFrame `df` with 'total_gross' and 'genres' columns

# Group by genre, calculate mean and total gross
genre_gross = df.groupby('genres')['total_gross'].agg(['mean', 'sum'])

# Bar Chart (Mean Gross)
plt.figure(figsize=(10, 6))
genre_gross['mean'].plot(kind='bar')
plt.title('Average Total Gross by Genre')
plt.ylabel('Average Total Gross (USD)')
plt.xticks(rotation=45)
plt.show()

# Pie Chart (Total Gross)
plt.figure(figsize=(8, 8))
genre_gross['sum'].plot(kind='pie', autopct='%1.1f%%')
plt.title('Total Gross Distribution by Genre')
plt.ylabel('')  # Remove y-label for pie chart
plt.show()
